## 1. Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import kagglehub
import os
import plotly.express as px
import nbformat
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

## 2. Data Loading

In [ ]:
path = kagglehub.dataset_download("akashdeepkuila/big-mart-sales")

print("Path to dataset files:", path)
print(os.listdir(path))
path = os.path.join(path, "Train-Set.csv")

100%|██████████| 307k/307k [00:00<00:00, 80.9MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/akashdeepkuila/big-mart-sales/versions/1
['Test-Set.csv', 'Train-Set.csv']


### 2.1 Dataset Structure and Summary

In [ ]:
df = pd.read_csv(path)

print("\nFirst 5 rows: ")
display(df.head())
print("\nLast 5 rows: ")
display(df.tail())

print(f"Dataset rows: {df.shape[0]}, Columns: {df.shape[1]}\n")

print("Columns:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2}. {col:<20}"
          f"Type: {str(df[col].dtype):<7}"
          f" | Missing: {df[col].isna().sum():<4} "
          f"({df[col].isnull().sum()/df.shape[0]*100:5.2f}%)"
          f" | Unique: {df[col].nunique():<7}"
          f" | duplicates: {df.duplicated().sum()}")


df.drop(["ProductID", "OutletID"], axis=1, inplace=True)



First 5 rows: 


,ProductID,Weight,FatContent,ProductVisibility,ProductType,MRP,OutletID,EstablishmentYear,OutletSize,LocationType,OutletType,OutletSales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052



Last 5 rows: 


,ProductID,Weight,FatContent,ProductVisibility,ProductType,MRP,OutletID,EstablishmentYear,OutletSize,LocationType,OutletType,OutletSales
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,FDS36,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,NaN,Tier 2,Supermarket Type1,549.2850
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976
8522,DRG01,14.800,Low Fat,0.044878,Soft Drinks,75.4670,OUT046,1997,Small,Tier 1,Supermarket Type1,765.6700


Dataset rows: 8523, Columns: 12

Columns:
   1. ProductID           Type: object  | Missing: 0    ( 0.00%) | Unique: 1559    | duplicates: 0
   2. Weight              Type: float64 | Missing: 1463 (17.17%) | Unique: 415     | duplicates: 0
   3. FatContent          Type: object  | Missing: 0    ( 0.00%) | Unique: 5       | duplicates: 0
   4. ProductVisibility   Type: float64 | Missing: 0    ( 0.00%) | Unique: 7880    | duplicates: 0
   5. ProductType         Type: object  | Missing: 0    ( 0.00%) | Unique: 16      | duplicates: 0
   6. MRP                 Type: float64 | Missing: 0    ( 0.00%) | Unique: 5938    | duplicates: 0
   7. OutletID            Type: object  | Missing: 0    ( 0.00%) | Unique: 10      | duplicates: 0
   8. EstablishmentYear   Type: int64   | Missing: 0    ( 0.00%) | Unique: 9       | duplicates: 0
   9. OutletSize          Type: object  | Missing: 2410 (28.28%) | Unique: 3       | duplicates: 0
  10. LocationType        Type: object  | Missing: 0    ( 0.00%) | 

### 2.2 Data Quality Check

In [ ]:
missing_count = df.isna().sum().sum()
if missing_count == 0:
    print("No missing values in the dataset.")
else:
    print(f"\n  Number of missing values in the dataset: {missing_count} in following columns:")
    for col in df.columns:
        if df[col].isna().sum() > 0:
            print(f"    {col}")

duplicate_count = df.duplicated().sum()
if duplicate_count == 0:
    print("\nNo duplicate rows in the dataset.")
else:
    print(f"\n  Number of duplicate rows in the dataset: {duplicate_count} in following rows:")
    duplicated_df = df[df.duplicated()]
    for col in duplicated_df.columns:
        print(f"    {col}")



  Number of missing values in the dataset: 3873 in following columns:
    Weight
    OutletSize

No duplicate rows in the dataset.


In [ ]:
cat_columns = df.select_dtypes(include=['object'])
for col in cat_columns:
    print(f"\nValue counts for column {df[col].value_counts(dropna=False)}:")



Value counts for column FatContent
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: count, dtype: int64:

Value counts for column ProductType
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: count, dtype: int64:

Value counts for column OutletSize
Medium    2793
NaN       2410
Small     2388
High       932
Name: count, dtype: int64:

Value counts for column LocationType
Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: count, dtype: int64:

Value counts for column OutletType
Supermarket Type1    5577
Grocery Store     

In [ ]:
num_summary = df.select_dtypes(include=['number']).describe()
print("\nStatistical summary of numerical columns:")
display(num_summary)


Statistical summary of numerical columns:


,Weight,ProductVisibility,MRP,EstablishmentYear,OutletSales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


## 3. Data Cleaning and Preprocessing

### 3.1 Column Renaming

In [ ]:
df.rename(columns={
    "OutletType": "StoreCategory",
    "LocationType": "CityTier",
    "OutletSize": "StoreSize",
    "OutletSales": "StoreSales"
}, inplace=True)

df["FatContent"] = df["FatContent"].replace({
    "LF": "Low Fat",
    "low fat": "Low Fat",
    "reg": "Regular",
})

print("\nUpdated fat content values:\n")
print(df["FatContent"].value_counts(dropna=False))


Updated fat content values:

FatContent
Low Fat    5517
Regular    3006
Name: count, dtype: int64


### 3.2 Missing Store Size Values Analysis

In [ ]:
display(pd.crosstab(df["StoreCategory"], df["StoreSize"], dropna=False))
display(pd.crosstab(df["CityTier"], df["StoreSize"], dropna=False))

size_counts = df.groupby(
    ["StoreCategory", "CityTier"],
)["StoreSize"].value_counts(dropna=False).unstack(fill_value=0)

display(size_counts)

agg_sales = df.groupby(
    ["StoreCategory", "CityTier", "StoreSize"],
    dropna=False
)["StoreSales"].agg(["count", "mean", "median"])

display(agg_sales)

StoreSize,High,Medium,Small,NaN
StoreCategory,,,,
Grocery Store,0,0,528,555
Supermarket Type1,932,930,1860,1855
Supermarket Type2,0,928,0,0
Supermarket Type3,0,935,0,0


StoreSize,High,Medium,Small,NaN
CityTier,,,,
Tier 1,0,930,1458,0
Tier 2,0,0,930,1855
Tier 3,932,1863,0,555


StoreSize                   High  Medium  Small   NaN
StoreCategory     CityTier                           
Grocery Store     Tier 1       0       0    528     0
                  Tier 3       0       0      0   555
Supermarket Type1 Tier 1       0     930    930     0
                  Tier 2       0       0    930  1855
                  Tier 3     932       0      0     0
Supermarket Type2 Tier 3       0     928      0     0
Supermarket Type3 Tier 3       0     935      0     0

count         mean     median
StoreCategory     CityTier StoreSize                               
Grocery Store     Tier 1   Small        528   340.329723   265.3213
                  Tier 3   NaN          555   339.351662   250.3408
Supermarket Type1 Tier 1   Medium       930  2348.354635  1966.1074
                           Small        930  2277.844267  1945.8005
                  Tier 2   Small        930  2438.841866  2109.2544
                           NaN         1855  2266.410119  1946.7992
                  Tier 3   High         932  2298.995256  2050.6640
Supermarket Type2 Tier 3   Medium       928  1995.498739  1655.1788
Supermarket Type3 Tier 3   Medium       935  3694.038558  3364.9532

In [ ]:
import plotly.express as px

df_plot = df.copy()
df_plot["StoreSize"] = df_plot["StoreSize"].fillna("Missing")

counts = (
    df_plot.groupby(["StoreCategory","CityTier","StoreSize"])
           .size().reset_index(name="Count")
)

# --- Bar chart ---
fig = px.bar(
    counts,
    x="StoreCategory", y="Count",
    color="StoreSize",
    facet_col="CityTier",
    barmode="group",
    color_discrete_sequence=px.colors.qualitative.Set2,
    category_orders={
        "CityTier": ["Tier 1","Tier 2","Tier 3"],
        "StoreCategory": ["Grocery Store","Supermarket Type1","Supermarket Type2","Supermarket Type3"],
        "StoreSize": ["Small","Medium","High","Missing"]
    },
    title="Value counts of Store Size by Store Category and City Tier"
)
fig.update_layout(
    width=1000, height=550,
    bargap=0.15,
    font=dict(color="black"),
    title_font=dict(size=22, color="black")
)
fig.show()

# --- Box chart ---
fig = px.box(
    df_plot[df_plot["StoreSize"].isin(["Small","Missing"])],
    x="StoreSize", y="StoreSales", color="StoreSize",
    color_discrete_sequence=px.colors.qualitative.Set2,
    title="Store Sales Distribution in Small vs Missing"
)
fig.update_layout(
    xaxis_title="Store Size",
    yaxis_title="Store Sales",
    font=dict(color="black"),
    title_font=dict(size=22, color="black")
)
fig.show()


### 3.3 Store Size Imputation

In [ ]:
df.loc[(df["StoreSize"].isna()) &
       (df["StoreCategory"] == "Grocery Store") &
       (df["CityTier"] == "Tier 3"), "StoreSize"] = "Small"

df.loc[(df["StoreSize"].isna()) &
       (df["StoreCategory"] == "Supermarket Type1") &
       (df["CityTier"] == "Tier 2"), "StoreSize"] = "Small"

print(df["StoreSize"].value_counts(dropna=False))


StoreSize
Small     4798
Medium    2793
High       932
Name: count, dtype: int64


### 3.4 Product Weight Analysis

In [ ]:
weight_mean = df["Weight"].mean()
weight_median = df["Weight"].median()
print(f"\nWeight total mean: {weight_mean:.2f}, total median: {weight_median:.2f}")

weight_stats = df.groupby("ProductType")["Weight"].agg(["count","mean","median","std"])
display(weight_stats.head())



Weight total mean: 12.86, total median: 12.60


,count,mean,median,std
ProductType,,,,
Baking Goods,536,12.277108,11.650,4.773622
Breads,204,11.346936,10.600,4.440540
Breakfast,89,12.768202,10.695,5.038131
Canned,539,12.305705,12.150,4.586564
Dairy,566,13.426069,13.350,4.686532


### 3.5 Weight Imputation

In [ ]:
df["Weight"] = df["Weight"].fillna(
    df.groupby("ProductType")["Weight"].transform("median")
)

### 3.6 Product Visibility Analysis

In [ ]:
visibility_mean = df["ProductVisibility"].mean()
visibility_median = df["ProductVisibility"].median()
print(f"Visibility total mean: {visibility_mean}, total median: {visibility_median}")

visibility_stats = df.groupby("ProductType")["ProductVisibility"].agg(["count","mean","median","std"])
display(visibility_stats.head())

Visibility total mean: 0.06613202877895107, total median: 0.053930934


,count,mean,median,std
ProductType,,,,
Baking Goods,648,0.069169,0.058725,0.052248
Breads,251,0.066255,0.055873,0.048816
Breakfast,110,0.085723,0.068893,0.052807
Canned,649,0.068129,0.050786,0.053730
Dairy,682,0.072427,0.063220,0.053205


### 3.7 Product Visibility Imputation

In [ ]:
df["ProductVisibility"] = df["ProductVisibility"].replace(0, np.nan)
df["ProductVisibility"] = df["ProductVisibility"].fillna(
    df.groupby("ProductType")["ProductVisibility"].transform("median")
)

## 4. Outlier Detection

In [ ]:
num_cols = ["Weight","MRP","ProductVisibility","StoreSales"]
for col in num_cols:
    fig = px.box(df, y=col,title=f"Boxplot of {col}")
    fig.show()
    fig = px.histogram(df, x=col, nbins=40, title=f"Histogram of {col}")
    fig.show()



### 4.1 IQR-Based Outlier Removal


In [ ]:
cols_to_check = ["ProductVisibility", "StoreSales"]
previous_rows = df.shape[0]
for col in cols_to_check:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]

    print(f"\nColumn: {col}")
    print(f"Q1: {Q1:.4f}, Q3: {Q3:.4f}, IQR: {IQR:.4f}")
    print(f"Lower Bound: {lower_bound:.4f}, Upper Bound: {upper_bound:.4f}")
    print(f"Number of outliers: {outliers.shape[0]}")
    print(f"Percentage: {100 * outliers.shape[0] / df.shape[0]:.2f}%")

    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]



Column: ProductVisibility
Q1: 0.0331, Q3: 0.0946, IQR: 0.0615
Lower Bound: -0.0592, Upper Bound: 0.1868
Number of outliers: 173
Percentage: 2.03%

Column: StoreSales
Q1: 882.8508, Q3: 3131.9232, IQR: 2249.0724
Lower Bound: -2490.7578, Upper Bound: 6505.5318
Number of outliers: 185
Percentage: 2.22%


### 4.2 Outlier Removal Summary


In [ ]:
print(f"\nRows before outlier removal: {previous_rows}, after: {df.shape[0]}")
print(f"Total rows removed: {previous_rows - df.shape[0]} ({100 * (previous_rows - df.shape[0]) / previous_rows:.2f}%)")


Rows before outlier removal: 8523, after: 8165
Total rows removed: 358 (4.20%)


## 5. Exploratory Data Analysis and Feature Engineering

### 5.1 Numerical Feature Correlations

In [ ]:
corr = df.select_dtypes(include=[np.number]).corr()
display(corr)

,Weight,ProductVisibility,MRP,EstablishmentYear,StoreSales
Weight,1.000000,-0.011316,0.028360,0.006120,0.010683
ProductVisibility,-0.011316,1.000000,-0.000404,-0.032210,-0.068752
MRP,0.028360,-0.000404,1.000000,0.026817,0.565211
EstablishmentYear,0.006120,-0.032210,0.026817,1.000000,-0.012391
StoreSales,0.010683,-0.068752,0.565211,-0.012391,1.000000


In [ ]:
corr = df.select_dtypes(include=[np.number]).corr()
fig = px.imshow(
    corr,
    text_auto=True,
    aspect="auto",
    color_continuous_scale="Viridis",
    title="Correlation Heatmap of Numerical Features"
)

fig.update_layout(
    width=800, height=600,
    font=dict(color="black"),
    title_font=dict(size=22, color="black"),
)
fig.show()

### 5.1 Categorical Feature Correlations

In [ ]:
cat_cols = ["StoreCategory", "CityTier", "StoreSize", "FatContent", "ProductType"]

for col in cat_cols:
    fig = px.box(
        df, x=col, y="StoreSales", color=col,
        title=f"StoreSales by {col}",
        color_discrete_sequence=px.colors.qualitative.Set2
    )
    fig.show()

### 5.2 Distribution Analysis
Examine the distribution of key variables to understand their characteristics.

In [ ]:
px.histogram(df,'ProductVisibility', nbins = 80)

In [ ]:
px.histogram(df,'StoreSales', nbins = 80)

### 5.3 Feature Engineering

In [ ]:
df['SqrtProductVisibility'] = np.sqrt(df['ProductVisibility'])

group_map = {
    "Fruits and Vegetables": "Fresh Produce",
    "Meat": "Fresh Produce",
    "Seafood": "Fresh Produce",

    "Snack Foods": "Snacks & Beverages",
    "Soft Drinks": "Snacks & Beverages",
    "Hard Drinks": "Snacks & Beverages",

    "Breads": "Grains & Bakery",
    "Baking Goods": "Grains & Bakery",
    "Breakfast": "Grains & Bakery",
    "Starchy Foods": "Grains & Bakery",

    "Dairy": "Dairy & Frozen",
    "Frozen Foods": "Dairy & Frozen",

    "Canned": "Canned & Packaged",
    "Others": "Canned & Packaged",

    "Household": "Household & Hygiene",
    "Health and Hygiene": "Household & Hygiene"
}

df["ProductGroup"] = df["ProductType"].map(group_map)


In [ ]:
px.histogram(df,'SqrtProductVisibility', nbins = 80)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8165 entries, 0 to 8522
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Weight                 8165 non-null   float64
 1   FatContent             8165 non-null   object 
 2   ProductVisibility      8165 non-null   float64
 3   ProductType            8165 non-null   object 
 4   MRP                    8165 non-null   float64
 5   EstablishmentYear      8165 non-null   int64  
 6   StoreSize              8165 non-null   object 
 7   CityTier               8165 non-null   object 
 8   StoreCategory          8165 non-null   object 
 9   StoreSales             8165 non-null   float64
 10  SqrtProductVisibility  8165 non-null   float64
 11  ProductGroup           8165 non-null   object 
 12  YearsSinceOpened       8165 non-null   int64  
dtypes: float64(5), int64(2), object(6)
memory usage: 893.0+ KB


In [ ]:
df.drop('ProductVisibility', axis = 1)

,Weight,FatContent,ProductType,MRP,EstablishmentYear,StoreSize,CityTier,StoreCategory,StoreSales,SqrtProductVisibility,ProductGroup,YearsSinceOpened
0,9.300,Low Fat,Dairy,249.8092,1999,Medium,Tier 1,Supermarket Type1,3735.1380,0.126678,Dairy & Frozen,14
1,5.920,Regular,Soft Drinks,48.2692,2009,Medium,Tier 3,Supermarket Type2,443.4228,0.138846,Snacks & Beverages,4
2,17.500,Low Fat,Meat,141.6180,1999,Medium,Tier 1,Supermarket Type1,2097.2700,0.129461,Fresh Produce,14
3,19.200,Regular,Fruits and Vegetables,182.0950,1998,Small,Tier 3,Grocery Store,732.3800,0.242443,Fresh Produce,15
4,8.930,Low Fat,Household,53.8614,1987,High,Tier 3,Supermarket Type1,994.7052,0.221397,Household & Hygiene,26
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,6.865,Low Fat,Snack Foods,214.5218,1987,High,Tier 3,Supermarket Type1,2778.3834,0.238293,Snacks & Beverages,26
8519,8.380,Regular,Baking Goods,108.1570,2002,Small,Tier 2,Supermarket Type1,549.2850,0.216754,Grains & Bakery,11
8520,10.600,Low Fat,Health and Hygiene,85.1224,2004,Small,Tier 2,Supermarket Type1,1193.1136,0.187580,Household & Hygiene,9
8521,7.210,Regular,Snack Foods,103.1332,2009,Medium,Tier 3,Supermarket Type2,1845.5976,0.381078,Snacks & Beverages,4


## 6 Visualizations

### 6.1 Product-Specific

In [ ]:
sales_pt = df.groupby("ProductType", as_index=False)["StoreSales"].sum().sort_values("StoreSales", ascending=False)

fig = px.bar(
    sales_pt, x="ProductType", y="StoreSales",
    title="Which Products Sell the Most?",
    text="StoreSales",
    color="ProductType",
    color_discrete_sequence=px.colors.qualitative.Set2
    )

fig.update_traces(
    texttemplate="%{text:.2s}",
    textposition="outside"
    )

fig.update_layout(
    width=900, height=500,
    xaxis_tickangle=45,
    yaxis_tickformat=".2s",
    margin=dict(l=40, r=40, t=60, b=120),
    showlegend=False
    )

fig.show()

sales_pg = df.groupby("ProductGroup", as_index=False)["StoreSales"].sum()
sales_pg["Share"] = sales_pg["StoreSales"] / sales_pg["StoreSales"].sum()

fig = px.pie(
    sales_pg, names="ProductGroup", values="StoreSales",
    title="Product Category Share of Total Sales",
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig.update_traces(
    texttemplate="%{label}<br>%{percent}",
    textposition="outside"
)
fig.update_layout(
    width=900, height=400,
    margin=dict(l=40, r=40, t=60, b=60)
)
fig.show()


sales_fc = df.groupby("FatContent", as_index=False)["StoreSales"].sum().sort_values("StoreSales", ascending=False)

fig = px.bar(
    sales_fc, x="FatContent", y="StoreSales",
    title="Sales Comparison by Low Fat vs Regular Products",
    text="StoreSales", color="FatContent",
    color_discrete_sequence=px.colors.qualitative.Set2
    )
fig.update_traces(texttemplate="%{text:.0f}")
fig.show()

### 6.2 Location-Specific

In [ ]:
# Enforce natural order for city tiers
tier_order = ["Tier 1", "Tier 2", "Tier 3"]
df["CityTier"] = pd.Categorical(df["CityTier"], categories=tier_order, ordered=True)

sales_tier = (
    df.groupby("CityTier", observed=True)["StoreSales"].sum().reset_index()
)

# Bar chart with formatted labels
fig = px.bar(
    sales_tier, x="CityTier", y="StoreSales",
    text=sales_tier["StoreSales"].apply(lambda x: f"{x:,.0f}"),
    color="CityTier",
    color_discrete_sequence=px.colors.qualitative.Set2,
    title="Which City Tier Generates the Highest Total Sales?"
)

fig.update_traces(textposition="outside", textfont_size=12)
fig.update_layout(
    yaxis_title="Total Sales",
    xaxis_title="City Tier",
    height=500, width=800,
    uniformtext_minsize=12,
    uniformtext_mode="hide"
)

fig.show()

In [ ]:
avg_cat = (
    df.groupby("StoreCategory", observed=True)["StoreSales"]
      .mean()
      .reset_index()
      .sort_values("StoreSales", ascending=False)
)

fig = px.scatter(
    avg_cat, x="StoreCategory", y="StoreSales",
    size=[10]*len(avg_cat), color="StoreCategory",
    text=avg_cat["StoreSales"].round(1),
    title="Which Supermarket Type Generates the Highest Average Sales?",
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig.update_layout(height=600)
fig.add_traces(px.line(avg_cat, x="StoreCategory", y="StoreSales").data)
fig.update_traces(textposition="top center")
fig.show()


### 6.3 Price and Visibility

In [ ]:
import plotly.express as px

df["Visibility_Bin"] = pd.qcut(df["ProductVisibility"], 4, labels=["Low", "Medium", "High", "Very High"])

avg_vis = df.groupby("Visibility_Bin")["StoreSales"].mean().reset_index()

fig = px.bar(
    avg_vis, x="Visibility_Bin", y="StoreSales",
    text_auto=".2s",
    color="Visibility_Bin",
    color_discrete_map={
        "Low": "#66c2a5",
        "Medium": "#fc8d62",
        "High": "#8da0cb",
        "Very High": "#e78ac3"
    },
    title="Does Visibility Affect Sales?"
)

fig.update_traces(textposition="outside")

fig.show()

df.drop("Visibility_Bin", axis=1, inplace=True)


/var/folders/8d/cbtkm7jj6zx2f7tj9clxr4w00000gn/T/ipykernel_8852/433401070.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [ ]:
df["MRP_Bin"] = pd.cut(df["MRP"], bins=20)

# Average sales per bin + use numeric midpoint for x
avg_price = df.groupby("MRP_Bin", observed=True)["StoreSales"].mean().reset_index()
avg_price["MRP_Mid"] = avg_price["MRP_Bin"].apply(lambda iv: iv.mid)  # numeric

fig = px.line(
    avg_price, x="MRP_Mid", y="StoreSales", markers=True,
    title="Average Sales by Price Range (MRP midpoints)",
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig.update_layout(xaxis_title="MRP (bin midpoint)")
fig.show()

# cleanup
df.drop("MRP_Bin", axis=1, inplace=True)

## 7. Feature Encoding and Data Preparation

In [ ]:
enc = LabelEncoder()
df["ProductType"] = enc.fit_transform(df["ProductType"])

categorical_cols = ["StoreSize", "CityTier", "StoreCategory", "FatContent", "ProductGroup"]
one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', one_hot_encoder, categorical_cols)
    ],
    remainder='passthrough'
)

df_encoded = preprocessor.fit_transform(df)

onehot_feature_names = preprocessor.named_transformers_['onehot'].get_feature_names_out(categorical_cols)
remaining_feature_names = [col for col in df.columns if col not in categorical_cols]

new_column_names = list(onehot_feature_names) + remaining_feature_names

df = pd.DataFrame(df_encoded, columns=new_column_names)

In [ ]:
df.head(20)

,StoreSize_High,StoreSize_Medium,StoreSize_Small,CityTier_Tier 1,CityTier_Tier 2,CityTier_Tier 3,StoreCategory_Grocery Store,StoreCategory_Supermarket Type1,StoreCategory_Supermarket Type2,StoreCategory_Supermarket Type3,...,ProductGroup_Grains & Bakery,ProductGroup_Household & Hygiene,ProductGroup_Snacks & Beverages,Weight,ProductVisibility,ProductType,MRP,EstablishmentYear,StoreSales,SqrtProductVisibility
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,9.300,0.016047,4.0,249.8092,1999.0,3735.1380,0.126678
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,5.920,0.019278,14.0,48.2692,2009.0,443.4228,0.138846
2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,17.500,0.016760,10.0,141.6180,1999.0,2097.2700,0.129461
3,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,19.200,0.058778,6.0,182.0950,1998.0,732.3800,0.242443
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,8.930,0.049017,9.0,53.8614,1987.0,994.7052,0.221397
5,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,10.395,0.062343,0.0,51.4008,2009.0,556.6088,0.249687
6,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,13.650,0.012741,13.0,57.6588,1987.0,343.5528,0.112876
7,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,13.150,0.127470,13.0,107.7622,1985.0,4022.7636,0.357029
8,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,16.200,0.016687,5.0,96.9726,2002.0,1076.5986,0.129179
9,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,19.200,0.094450,5.0,187.8214,2007.0,4710.5350,0.307327


In [ ]:
df.describe()

,StoreSize_High,StoreSize_Medium,StoreSize_Small,CityTier_Tier 1,CityTier_Tier 2,CityTier_Tier 3,StoreCategory_Grocery Store,StoreCategory_Supermarket Type1,StoreCategory_Supermarket Type2,StoreCategory_Supermarket Type3,Weight,FatContent,ProductVisibility,ProductType,MRP,EstablishmentYear,StoreSales,SqrtProductVisibility
count,8165.000000,8165.000000,8165.000000,8165.000000,8165.000000,8165.000000,8165.000000,8165.000000,8165.000000,8165.000000,8165.000000,8165.000000,8165.000000,8165.000000,8165.000000,8165.000000,8165.000000,8165.000000
mean,0.112309,0.327495,0.560196,0.279118,0.335334,0.385548,0.112186,0.672750,0.113166,0.101898,12.828624,0.352113,0.066369,7.234415,139.152658,1998.102143,2095.030417,0.243782
std,0.315765,0.469328,0.496394,0.448593,0.472136,0.486754,0.315615,0.469238,0.316815,0.302533,4.283638,0.477658,0.042846,4.205135,61.578057,8.286640,1501.287502,0.083306
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.555000,0.000000,0.003575,0.000000,31.290000,1985.000000,33.290000,0.059789
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.300000,0.000000,0.032610,4.000000,93.043600,1987.000000,865.540000,0.180582
50%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,12.850000,0.000000,0.057584,6.000000,141.115400,1999.000000,1789.670400,0.239966
75%,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,16.100000,1.000000,0.092158,10.000000,183.692400,2004.000000,3025.395200,0.303576
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,21.350000,1.000000,0.186826,15.000000,266.888400,2009.000000,6503.534400,0.432234


In [ ]:
print(df.dtypes)

StoreSize_High                     float64
StoreSize_Medium                   float64
StoreSize_Small                    float64
CityTier_Tier 1                    float64
CityTier_Tier 2                    float64
CityTier_Tier 3                    float64
StoreCategory_Grocery Store        float64
StoreCategory_Supermarket Type1    float64
StoreCategory_Supermarket Type2    float64
StoreCategory_Supermarket Type3    float64
Weight                             float64
FatContent                         float64
ProductVisibility                  float64
ProductType                        float64
MRP                                float64
EstablishmentYear                  float64
StoreSales                         float64
SqrtProductVisibility              float64
dtype: object


### 8 Linear Regression Model

In [ ]:
X = df.drop("StoreSales", axis=1)
y = df["StoreSales"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

model = sm.OLS(y_train, X_train).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             StoreSales   R-squared:                       0.553
Model:                            OLS   Adj. R-squared:                  0.552
Method:                 Least Squares   F-statistic:                     575.9
Date:                Fri, 19 Sep 2025   Prob (F-statistic):               0.00
Time:                        22:03:59   Log-Likelihood:                -54405.
No. Observations:                6532   AIC:                         1.088e+05
Df Residuals:                    6517   BIC:                         1.089e+05
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const     